In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path and version numbers for python depend on operating system
!which python
!python --version

In [ ]:
# environment variable
%env PYTHONPATH=

In [ ]:
# install virtual environment package
!pip install virtualenv

In [ ]:
# create virtual environment
!virtualenv llm

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

In [ ]:
!chmod +x Miniconda3-latest-Linux-x86_64.sh

In [ ]:
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
!pip install chardet

In [ ]:
!conda install -q -y --prefix /usr/local python=3.10 ujson

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages/')

In [ ]:
# activate conda enviornment
import os
os.environ['CONDA_PREFIX'] = '/usr/local/envs/llm'

In [ ]:
%cd /content/drive/MyDrive/ColabNotebooks/2023ML/Lab4/Chinese-LLaMA-Alpaca-2-main/Chinese-LLaMA-Alpaca-2-main
!pip install -r requirements.txt

In [ ]:
import torch
print(torch.cuda.is_available())
torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install datasets

In [ ]:
!pip3 install deepspeed

In [ ]:
!pip install scipy

In [ ]:
!pip install accelerate

In [ ]:
!pip install pydantic==1.10.9

In [ ]:
# 以下範例為輸入json用以批次生成的腳本，並將結果存於 ./answer.csv
# 你也可以自行修改腳本，整理成自己習慣的資料格式(txt..)用於批次輸入
import torch
import os
import argparse
import json,csv
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    AutoTokenizer,
    StoppingCriteria,
    BitsAndBytesConfig
)

# 訓練時System Prompt可以自訂，生成時建議與訓練的Prompt一致
# 請參考script/training/build_dataset.py 進行Prompt的調整
DEFAULT_SYSTEM_PROMPT = """你是一個樂於助人的助手。"""

TEMPLATE_WITH_SYSTEM_PROMPT = (
    "[INST] <<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n{instruction} [/INST]"
)

parser = argparse.ArgumentParser()
parser.add_argument(
    '--base_model',
    default='/content/drive/MyDrive/ColabNotebooks/2023ML/Lab4/Chinese-LLaMA-Alpaca-2-main/Chinese-LLaMA-Alpaca-2-main/llama-2-13b-combined-test',
    type=str,
    #required=True,
    help='Base model path')
parser.add_argument(
    '--gpus',
    default="0",
    type=str,
    help='If None, cuda:0 will be used. Inference using multi-cards: --gpus=0,1,... ')
parser.add_argument(
    '--load_in_8bit',
    action='store_true',
    help='Use 8 bit quantized model')
parser.add_argument(
    '--load_in_4bit',
    action='store_true',
    help='Use 4 bit quantized model')
args = parser.parse_args(args=[])

os.environ["CUDA_VISIBLE_DEVICES"] = args.gpus

# Get GPU devices
DEV = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Model
model = LlamaForCausalLM.from_pretrained(
    args.base_model,
    torch_dtype=torch.float16,
    #low_cpu_mem_usage=True,
    #device_map='auto',
    #quantization_config=BitsAndBytesConfig(
        #load_in_4bit=args.load_in_4bit,
        #load_in_8bit=args.load_in_8bit,
        #bnb_4bit_compute_dtype=torch.float16
    #)
)

# Load Tokenizer
tokenizer = LlamaTokenizer.from_pretrained(args.base_model)

# Do inference
with open('/content/drive/MyDrive/ColabNotebooks/2023ML/Lab4/AI1000.json', 'r') as file:
    json_data = json.load(file)
    with open('/content/drive/MyDrive/ColabNotebooks/2023ML/Lab4/answer.csv', 'w', newline='', encoding='utf-8') as csv_file:
        writer=csv.writer(csv_file)
        writer.writerow(['ID', 'Answer'])
        idx=1
        for row in json_data:
            id = idx
            instruction = row['instruction'] + '\n' + row['input']

            prompt = TEMPLATE_WITH_SYSTEM_PROMPT.format_map({'instruction': instruction,'system_prompt': DEFAULT_SYSTEM_PROMPT})
            inputs = tokenizer.encode(prompt+'\n', return_tensors="pt").to(DEV)

            generate_kwargs = dict(
                input_ids=inputs,
                temperature=0.2,
                top_p=0.9,
                top_k=40,
                do_sample=True,
                max_new_tokens=1, #為了回答選擇題而設定1
                repetition_penalty=1.1,
                guidance_scale=1.0
            )
            outputs = model.to(DEV).generate(**generate_kwargs)
            result = tokenizer.decode(outputs[0])
            print(result)
            response = result.split('[/INST]\n')[-1]
            writer.writerow([id, response[0]])
            idx=idx+1